In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy
# import functools
# import json
# import math
# import pandas as pd
# import pennylane as qml
# import pennylane.numpy as np
# import scipy

#### Quantum FOurier Challenge

In [8]:
def fourier_squared_distance(list_of_coeffs, param_list):
    """
    Returns the squared l2-distance in Fourier space between a function
    characterized by its Fourier coefficients and the output of the
    quantum model

    Args:
        list_of coeffs (list(float)): A list of seven coefficients
                                      corresponding to the Fourier
                                      coefficients of the function we
                                      want to approximate
        param_list (list(float)): A list of six parameters characterizing
                                  the angles in the trainable circuit.

    Returns: (float): Squared l2-distance between the given function
                      and the output of the quantum model
    """

    dev = qml.device("default.qubit", wires=3)

    # Feel free to define any helper functions, such as subcircuits, here.


    def entanglers(params):
        """ params = shape (3,1) array"""

        for i, param in enumerate(params): qml.RX(param, wires=[i])
        qml.CNOT(wires= [0,1])
        qml.CNOT(wires= [1,2])
        qml.CNOT(wires= [2,1])
    
    def encoder(x):
        for i in range(3):
            qml.RX(x, wires=[i])

    @qml.qnode(dev)
    def circuit(param_list, x):
        """This circuit returns the PauliZ expectation of
        the quantum model in the statement"""

        entanglers(param_list[:3])

        encoder(x)

        entanglers(param_list[3:])

        return qml.expval(qml.PauliZ(0))
    
    def fourier_coefficients(f,t, K):
        """
        Computes the first 2*K+1 Fourier coefficients of a 2*pi periodic function.
        """
        n_coeffs = 2 * K + 1
        
        y = np.fft.rfft(f(t)) / t.size
        return y
    
    t = np.linspace(0, 2 * np.pi, 7, endpoint=False)
    def f(x):
        return np.array([circuit(param_list, x_) for x_ in x])
    
    calc_coefficients = fourier_coefficients(f,t, 3)


    l2_norm = lambda c1,c2 :  np.sum([ np.abs(c1[j] - c2[j])**2 for j in range(len(list_of_coeffs)) ])   

    # Write a function that calculates the squared l2-distance here
    print(list_of_coeffs)
    print(calc_coefficients)
    
    return l2_norm(list_of_coeffs, calc_coefficients)
    # Return your final answer here

In [10]:
dev = qml.device("default.qubit", wires=3)

# Feel free to define any helper functions, such as subcircuits, here.


def entanglers(params):
    """ params = shape (3,1) array"""

    for i, param in enumerate(params): qml.RX(param, wires=[i])
    qml.CNOT(wires= [0,1])
    qml.CNOT(wires= [1,2])
    qml.CNOT(wires= [2,1])

def encoder(x):
    for i in range(3):
        qml.RX(x, wires=[i])

@qml.qnode(dev)
def circuit(param_list, x):
    """This circuit returns the PauliZ expectation of
    the quantum model in the statement"""

    entanglers(param_list[:3])

    encoder(x)

    entanglers(param_list[3:])

    return qml.expval(qml.PauliZ(0))

def fourier_coefficients(f,t, K):
    """
    Computes the first 2*K+1 Fourier coefficients of a 2*pi periodic function.
    """
    n_coeffs = 2 * K + 1
    
    y = np.fft.rfft(f(t)) / t.size
    return y

param_list = [2,2,2,3,4,5]
t = np.linspace(0, 2 * np.pi, 7, endpoint=False)
[circuit(param_list, x_) for x_ in t]

[tensor(0.41198225, requires_grad=True),
 tensor(0.30278107, requires_grad=True),
 tensor(-0.03442043, requires_grad=True),
 tensor(-0.34570264, requires_grad=True),
 tensor(-0.39666371, requires_grad=True),
 tensor(-0.14892892, requires_grad=True),
 tensor(0.21095239, requires_grad=True)]

In [9]:
fourier_squared_distance([-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5])

[-0.112422548, 0.0, 0.094790994, 0.0, 0.0, 0.094790994, 0.0]
[1.58603289e-17+0.00000000e+00j 2.05991123e-01-2.93633225e-02j
 0.00000000e+00-3.96508223e-17j 1.58603289e-17-9.91270558e-19j]


IndexError: index 4 is out of bounds for axis 0 with size 4

In [6]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = fourier_squared_distance(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated squared distance isn't quite right."

test_cases = [['[[-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5]]', '0.0036766085933034303'], ['[[-2.51161988e-01, 0.0, 1.22546112e-01, 0.0, 0.0,  1.22546112e-01, 0.0],[1.1,0.3,0.4,0.6,0.8,0.9]]', '0.6538589174369286']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correc-t!")

Running test case 0 with input '[[-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5]]'...
Runtime Error. name 'json' is not defined
Running test case 1 with input '[[-2.51161988e-01, 0.0, 1.22546112e-01, 0.0, 0.0,  1.22546112e-01, 0.0],[1.1,0.3,0.4,0.6,0.8,0.9]]'...
Runtime Error. name 'json' is not defined


#### CNOT universality challenge

In [100]:
def hadamard(wire):
    qml.U3(np.pi, 0 ,np.pi, wires= [wire] )
    qml.U3(np.pi/2, np.pi, np.pi, wires= [wire])
    
    
    

def cz(wires):
    hadamard(wires[1])
    qml.CNOT(wires = wires)
    hadamard(wires[1])
    
def circuit():
    """
    Succession of gates that will generate the requested matrix.
    This function does not receive any arguments nor does it return any values.
    """

    qml.U3(np.pi,0,np.pi, wires= [0])
    cz([0,1])
    qml.U3(np.pi,0,np.pi, wires= [0])
    hadamard(2)
    

    

    # Put your solution here ...
    # You only have to put U3 or CNOT gates
    

In [101]:
np.set_printoptions(precision=3)
a = qml.matrix(circuit)
s = a()
s = np.real(s)
s[np.abs(s) < 0.00001] = 0
print(s)

[[ 0.707  0.707  0.     0.     0.     0.     0.     0.   ]
 [ 0.707 -0.707  0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    -0.707 -0.707  0.     0.     0.     0.   ]
 [ 0.     0.    -0.707  0.707  0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.707  0.707  0.     0.   ]
 [ 0.     0.     0.     0.     0.707 -0.707  0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.707  0.707]
 [ 0.     0.     0.     0.     0.     0.     0.707 -0.707]]


In [24]:
import seaborn as sns

In [7]:
op = qml.RX(0.54, wires=0)
qml.matrix(op)

array([[0.9637709+0.j        , 0.       -0.26673144j],
       [0.       -0.26673144j, 0.9637709+0.j        ]])

In [16]:
def circuitt():
    qml.RX(np.pi, wires=1)
    qml.PauliZ(wires=0)
matrix_fn = qml.matrix(circuitt)
# theta = np.pi / 4
matrix_fn()

array([[ 6.123234e-17+0.j,  0.000000e+00+0.j,  0.000000e+00-1.j,
         0.000000e+00+0.j],
       [ 0.000000e+00+0.j, -6.123234e-17+0.j,  0.000000e+00+0.j,
         0.000000e+00+1.j],
       [ 0.000000e+00-1.j,  0.000000e+00+0.j,  6.123234e-17+0.j,
         0.000000e+00+0.j],
       [ 0.000000e+00+0.j,  0.000000e+00+1.j,  0.000000e+00+0.j,
        -6.123234e-17+0.j]])

In [17]:
matrix_fn()

array([[ 6.123234e-17+0.j,  0.000000e+00+0.j,  0.000000e+00-1.j,
         0.000000e+00+0.j],
       [ 0.000000e+00+0.j, -6.123234e-17+0.j,  0.000000e+00+0.j,
         0.000000e+00+1.j],
       [ 0.000000e+00-1.j,  0.000000e+00+0.j,  6.123234e-17+0.j,
         0.000000e+00+0.j],
       [ 0.000000e+00+0.j,  0.000000e+00+1.j,  0.000000e+00+0.j,
        -6.123234e-17+0.j]])

In [18]:
qml.matrix(circuitt)

<function pennylane.transforms.op_transforms.op_transform._create_wrapper.<locals>.wrapper(*args, **kwargs)>